In [1]:
pip install pandas sqlalchemy "oracledb>=2.0"

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------

In [2]:
import pandas as pd
from sqlalchemy import create_engine

# IMPORTANT : on passe user/password/DSN via connect_args (évite les soucis d’URL encoding avec le #)
engine = create_engine(
    "oracle+oracledb://",
    connect_args={
        "user": "raw_layer",
        "password": "Raw#123",
        "dsn": "localhost:1521/XEPDB1",   # host:port/service_name
    },
    pool_pre_ping=True,  # optionnel : vérifie la connexion
)

# Test rapide : lister les tables du schéma courant
pd.read_sql("SELECT table_name FROM user_tables ORDER BY table_name", engine)

,table_name
0,BRANCHES
1,CARS
2,CAR_CATEGORIES
3,CAR_TELEMETRY
4,CLIENTS
5,DAMAGES
6,IOT_ALERTS
7,IOT_DATA
8,IOT_DEVICES
9,MAINTENANCE


In [3]:
branches = pd.read_sql("SELECT * FROM Branches", engine)
cars     = pd.read_sql("SELECT * FROM Cars", engine)
devices  = pd.read_sql("SELECT * FROM IoT_Devices", engine)
cats     = pd.read_sql("SELECT * FROM Car_Categories", engine)

# Exemple de vue
avail = pd.read_sql("SELECT * FROM Available_Cars_With_IoT", engine)
avail.head()


,car_id,license_plate,brand,model,year,category_name,daily_rate,branch_name,city,device_serial,device_type,device_status
0,47,FES-406,Lexus,ES,2020,Luxury,780.0,Fes Medina,Fes,IOT-MA-0046,Fuel Monitor,Active
1,48,FES-407,Tesla,Model 3,2020,Electric,350.0,Fes Medina,Fes,IOT-MA-0047,GPS,Active
2,5,CMA-004,Tesla,Model 3,2024,Electric,350.0,Casablanca HQ,Casablanca,IOT-MA-0004,GPS,Maintenance
3,13,RBA-102,Lexus,ES,2018,Luxury,780.0,Rabat Agdal,Rabat,IOT-MA-0012,Fuel Monitor,Maintenance
4,19,RBA-108,Renault,Twingo,2025,Economy,220.0,Rabat Agdal,Rabat,IOT-MA-0018,GPS,Active


In [4]:
avail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_id         15 non-null     int64  
 1   license_plate  15 non-null     object 
 2   brand          15 non-null     object 
 3   model          15 non-null     object 
 4   year           15 non-null     int64  
 5   category_name  15 non-null     object 
 6   daily_rate     15 non-null     float64
 7   branch_name    15 non-null     object 
 8   city           15 non-null     object 
 9   device_serial  15 non-null     object 
 10  device_type    15 non-null     object 
 11  device_status  15 non-null     object 
dtypes: float64(1), int64(2), object(9)
memory usage: 1.5+ KB
